In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

# RDD만을 이용한 데이터 추출

In [2]:

movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [3]:
# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [4]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.take(5)

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [5]:
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()


[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [6]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [7]:
sc.stop()

In [8]:
# Spark SQL 사용해 보기

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [10]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [11]:
#스키마를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

In [12]:
# 2. 데이터 프레임 만들기

In [13]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [14]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [15]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [16]:
df.select("company").show()

+-------+
|company|
+-------+
|   마블|
|     DC|
|     DC|
| 디즈니|
|   마블|
+-------+



In [17]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [18]:
df.select(["year", "month", "day"]).show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2012|    4| 26|
|2013|    6| 13|
|2008|    8|  6|
|2014|    1| 16|
|2008|    4| 30|
+----+-----+---+



In [19]:
df.select(["year", "month", "day"]).filter(df.year >= 2013).show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2013|    6| 13|
|2014|    1| 16|
+----+-----+---+



In [20]:
df.filter((df.company =='마블') | (df.company == 'DC')).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [21]:
df.createOrReplaceTempView("movies") # 뷰의 이름

In [22]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [23]:
# 2010년 이후에 개봉한 영화를 조회

query = """
SELECT *
  FROM movies
  WHERE year > 2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [24]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """
SELECT name, company
  FROM movies
  WHERE year < 2014
"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [25]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """
    SELECT name, company, year
    FROM movies
    WHERE name like '%맨'
"""
spark.sql(query).show()

+--------+-------+----+
|    name|company|year|
+--------+-------+----+
|  슈퍼맨|     DC|2013|
|  배트맨|     DC|2008|
|아이언맨|   마블|2008|
+--------+-------+----+



In [26]:

# BETWEEN 특정 데이터와 데이터 사이를 조회

# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8
query = """
    SELECT name, company, year, month
    FROM movies
    WHERE month between '4' and '8'
"""
spark.sql(query).show()

+--------+-------+----+-----+
|    name|company|year|month|
+--------+-------+----+-----+
|어벤져스|   마블|2012|    4|
|  슈퍼맨|     DC|2013|    6|
|  배트맨|     DC|2008|    8|
|아이언맨|   마블|2008|    4|
+--------+-------+----+-----+



In [27]:
# Join 구현하기

In [28]:

attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [29]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [30]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [31]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [32]:
att_df.createOrReplaceTempView("att")

In [33]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [34]:
# df 와 join
query = '''
select movies.id, movies.name, movies.company, att.att
from movies
join att ON movies.id = att.id
'''
spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



In [35]:
# 데이터 프레임 API

In [36]:
# select
df.select("*").collect()

[Row(id=1, name='어벤져스', company='마블', year=2012, month=4, day=26),
 Row(id=2, name='슈퍼맨', company='DC', year=2013, month=6, day=13),
 Row(id=3, name='배트맨', company='DC', year=2008, month=8, day=6),
 Row(id=4, name='겨울왕국', company='디즈니', year=2014, month=1, day=16),
 Row(id=5, name='아이언맨', company='마블', year=2008, month=4, day=30)]

In [37]:
df.select("name", "company").collect()

[Row(name='어벤져스', company='마블'),
 Row(name='슈퍼맨', company='DC'),
 Row(name='배트맨', company='DC'),
 Row(name='겨울왕국', company='디즈니'),
 Row(name='아이언맨', company='마블')]

In [38]:
df.select(df.name, (df.year-2000).alias("year")).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|  12|
|  슈퍼맨|  13|
|  배트맨|   8|
|겨울왕국|  14|
|아이언맨|   8|
+--------+----+



In [39]:
# agg : Aggreagte의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id": "count"}).collect()

[Row(count(id)=5)]

In [40]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

[Row(min(year)=2008)]

In [41]:
df.groupBy().avg().collect()

[Row(avg(id)=3.0, avg(year)=2011.0, avg(month)=4.6, avg(day)=18.2)]

In [42]:
# 회사별 개봉월의 평균
df.groupBy('company').agg({"month": "mean"}).collect()

[Row(company='디즈니', avg(month)=1.0),
 Row(company='마블', avg(month)=4.0),
 Row(company='DC', avg(month)=7.0)]

In [43]:
# 회사 별 월 별 영화 개수 정보


In [44]:
# join : 다른 데이터 프레임과 사용자가 지정한 컬럼을 기준으로 합치는 작업
df.join(att_df, 'id').select(df.name, att_df.att).show()

+--------+-----------+
|    name|        att|
+--------+-----------+
|어벤져스|1.3934592E7|
|  슈퍼맨|  2182227.0|
|  배트맨|  4226242.0|
|겨울왕국|1.0303058E7|
|아이언맨|  4300365.0|
+--------+-----------+



In [45]:
# select, where, orderBy 절 사용
marvel_df = df.select("name", "company", "year").where("company=='마블'").orderBy("id")
marvel_df.collect()

[Row(name='어벤져스', company='마블', year=2012),
 Row(name='아이언맨', company='마블', year=2008)]

In [46]:
spark.stop()
sc.stop()

In [47]:
# SQL 최적화

In [48]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [49]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [50]:
# inferSchema : 자동으로 스키마 예측하게 하기
data = spark.read.csv(trip_file, inferSchema=True, header=True)

In [51]:
data.createOrReplaceTempView("mobility_data")

In [52]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [53]:
# 스파크 SQL을 사용하는 이유

In [54]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [55]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#469, 'count(1) AS trips#470]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#396 as string),  , -1)[0]], [split(cast(pickup_datetime#396 as string),  , -1)[0] AS pickup_date#469, count(1) AS trips#470L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#394,dispatching_base_num#395,pickup_datetime#396,dropoff_datetime#397,PULocationID#398,DOLocationID#399,SR_Flag#400])
      +- Relation [hvfhs_license_num#394,dispatching_base_num#395,pickup_datetime#396,dropoff_datetime#397,PULocationID#398,DOLocationID#399,SR_Flag#400] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#474], [_groupingexpression#474 AS pickup_date#469, count(1) AS trips#470L]
+- Project [split(cast(pickup_datetime#396 as string),  , -1)[0] AS _groupingexpression#474]
   +- Rel

In [56]:
#### 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#478]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#477]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#477], [pickup_date#477, count(1) AS trips#478L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#396 as string),  , -1)[0] AS pickup_date#477]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#394,dispatching_base_num#395,pickup_datetime#396,dropoff_datetime#397,PULocationID#398,DOLocationID#399,SR_Flag#400])
            +- Relation [hvfhs_license_num#394,dispatching_base_num#395,pickup_datetime#396,dropoff_datetime#397,PULocationID#398,DOLocationID#399,SR_Flag#400] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#477], [pickup_date#477, count(1) AS tri

# 택시 데이터 실습

In [57]:
from pyspark.sql.functions import first, last, min, max, sum, avg, count, expr

In [58]:
# 운행 데이터 프레임 생성, zone 데이터 프레임 생성
trip_data = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/fhvhv_tripdata_2020-03.csv')
zone_data = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/taxi+_zone_lookup.csv')

In [59]:
trip_data.show(1)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 1 row



In [60]:
zone_data.show(1)

+----------+-------+--------------+------------+
|LocationID|Borough|          Zone|service_zone|
+----------+-------+--------------+------------+
|         1|    EWR|Newark Airport|         EWR|
+----------+-------+--------------+------------+
only showing top 1 row



In [61]:
# 승차 Location(PULocationID)별 개수 세기
# 하차 Location(DOLocationID)별 개수 세기
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
#운송사별 운행 건수 비교
#승차 위치 Borough별 운행 건수
#서비스 존별 승차/하차 건수

In [62]:
# 승차 Location(PULocationID)별 개수 세기
trip_data.groupby('PULocationID').agg(count('*')).show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|         148|  116205|
|         243|   87431|
|          31|    5285|
|         137|   85552|
|          85|   46120|
|         251|    9080|
|          65|   66622|
|         255|  113947|
|          53|   17571|
|         133|   27200|
|          78|   76155|
|         108|   20378|
|         155|   39527|
|         211|   61075|
|         193|   20111|
|          34|   11823|
|         115|   10806|
|         126|   52833|
|         101|    8983|
|          81|   41425|
+------------+--------+
only showing top 20 rows



In [63]:
# 하차 Location(DOLocationID)별 개수 세기
trip_data.groupby('DOLocationID').agg(expr('count(*)')).show()

+------------+--------+
|DOLocationID|count(1)|
+------------+--------+
|         148|   91601|
|         243|   86795|
|          31|    5526|
|          85|   44509|
|         137|   80098|
|         251|    8525|
|          65|   58888|
|         255|  105051|
|          53|   19013|
|         133|   27760|
|          78|   74447|
|         155|   42239|
|         108|   21354|
|         211|   54176|
|         193|   19104|
|          34|   12392|
|         115|    9809|
|         101|    7218|
|         126|   59027|
|          81|   38445|
+------------+--------+
only showing top 20 rows



In [64]:
from pyspark.sql.functions import rank, desc, round
from pyspark.sql.window import Window

In [65]:
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
ndf = trip_data.groupBy('PULocationID','hvfhs_license_num').agg(expr('count(*) as ct'))
windowspec = Window.orderBy(desc('ct'))
AllRank = rank().over(windowspec)
ndf.withColumn('count_rank', AllRank).filter(ndf['hvfhs_license_num']=='HV0003').select('hvfhs_license_num','ct', 'count_rank').show(5)

+-----------------+------+----------+
|hvfhs_license_num|    ct|count_rank|
+-----------------+------+----------+
|           HV0003|163091|         1|
|           HV0003|134198|         2|
|           HV0003|114179|         3|
|           HV0003|112017|         4|
|           HV0003|110150|         5|
+-----------------+------+----------+
only showing top 5 rows



In [66]:
#운송사별 운행 건수 비교
ndf = trip_data.groupBy('hvfhs_license_num').agg(expr('count(*) as ct'))
windowspec = Window.orderBy(desc('ct'))
AllRank = rank().over(windowspec)
ndf.withColumn('count_rank', AllRank).select('hvfhs_license_num','ct', 'count_rank').show(5)

+-----------------+-------+----------+
|hvfhs_license_num|     ct|count_rank|
+-----------------+-------+----------+
|           HV0003|9836763|         1|
|           HV0005|3219535|         2|
|           HV0004| 336606|         3|
+-----------------+-------+----------+



In [67]:
#승차 위치 Borough별 운행 건수
trip_data.createOrReplaceTempView("trip_view")
zone_data.createOrReplaceTempView("zone_view")
query = '''
    select trip_view.PULocationID, zone_view.Borough, count (*)
    from trip_view
    join zone_view
    on trip_view.PULocationID = zone_view.LocationID
    group by trip_view.PULocationID, zone_view.Borough
'''
spark.sql(query).show()

+------------+-------------+--------+
|PULocationID|      Borough|count(1)|
+------------+-------------+--------+
|         215|       Queens|   44136|
|          32|        Bronx|   42128|
|         232|    Manhattan|   64543|
|          34|     Brooklyn|   11823|
|         235|        Bronx|   83347|
|         168|        Bronx|   98424|
|         260|       Queens|   52982|
|         118|Staten Island|   12375|
|          88|    Manhattan|   26235|
|          80|     Brooklyn|   98314|
|          50|    Manhattan|   80762|
|          92|       Queens|   54928|
|         216|       Queens|   77366|
|         122|       Queens|   16043|
|           9|       Queens|   10448|
|         117|       Queens|   18570|
|         186|    Manhattan|   79729|
|         107|    Manhattan|  102656|
|          62|     Brooklyn|   57126|
|         191|       Queens|   33139|
+------------+-------------+--------+
only showing top 20 rows



In [68]:
#서비스 존별 승차/하차 건수
query = '''
    select zone_view.service_zone, trip_view.PULocationID, count (*)
    from zone_view
    join trip_view
    on trip_view.PULocationID = zone_view.LocationID
    group by zone_view.service_zone, trip_view.PULocationID
'''
spark.sql(query).show()

+------------+------------+--------+
|service_zone|PULocationID|count(1)|
+------------+------------+--------+
|   Boro Zone|         198|   82045|
|   Boro Zone|         136|   48398|
|   Boro Zone|         240|    8457|
|   Boro Zone|          89|  124289|
|   Boro Zone|          81|   41425|
|   Boro Zone|          31|    5285|
|   Boro Zone|          22|   46127|
|   Boro Zone|          11|   15737|
|   Boro Zone|          39|  116669|
|   Boro Zone|         241|   65961|
|   Boro Zone|         154|    1242|
|   Boro Zone|         147|   40262|
| Yellow Zone|         162|  104300|
|   Boro Zone|          84|    5235|
|   Boro Zone|         189|   45973|
|   Boro Zone|          78|   76155|
|   Boro Zone|          17|  126228|
|   Boro Zone|         196|   32282|
|   Boro Zone|         243|   87431|
|   Boro Zone|          15|    7685|
+------------+------------+--------+
only showing top 20 rows



In [69]:
query = '''
    select zone_view.service_zone, trip_view.DOLocationID, count (*)
    from zone_view
    join trip_view
    on trip_view.DOLocationID = zone_view.LocationID
    group by zone_view.service_zone, trip_view.DOLocationID
'''
spark.sql(query).show()

+------------+------------+--------+
|service_zone|DOLocationID|count(1)|
+------------+------------+--------+
|   Boro Zone|         136|   48554|
|   Boro Zone|         198|   84361|
|   Boro Zone|         240|    8697|
|   Boro Zone|          89|  126958|
|   Boro Zone|          81|   38445|
|   Boro Zone|          31|    5526|
|   Boro Zone|          22|   46493|
|   Boro Zone|          11|   15403|
|   Boro Zone|          39|  113966|
|   Boro Zone|         241|   67266|
|   Boro Zone|         154|    1462|
|   Boro Zone|         189|   46100|
| Yellow Zone|         162|   95681|
|   Boro Zone|         147|   38744|
|   Boro Zone|          78|   74447|
|   Boro Zone|          84|    5100|
|   Boro Zone|          17|  128152|
|   Boro Zone|         196|   29948|
|   Boro Zone|         243|   86795|
|   Boro Zone|          15|    8059|
+------------+------------+--------+
only showing top 20 rows



# 추가 퀴즈 3문제

## 1. 승차 위치의 zone, borough 그리고 승차 건수를 출력 + 승차건수가 많은 순으로 상위 10개만 출력

In [71]:
query = '''
    select trip_view.PULocationID, zone_view.service_zone, zone_view.Borough, count (*) as ct
    from trip_view
    join zone_view
    on trip_view.PULocationID = zone_view.LocationID
    group by trip_view.PULocationID, zone_view.service_zone, zone_view.Borough
    order by ct desc
    limit 10
'''
spark.sql(query).show()

+------------+------------+---------+------+
|PULocationID|service_zone|  Borough|    ct|
+------------+------------+---------+------+
|          61|   Boro Zone| Brooklyn|222094|
|          79| Yellow Zone|Manhattan|183821|
|          76|   Boro Zone| Brooklyn|168311|
|         132|    Airports|   Queens|163734|
|         138|    Airports|   Queens|155876|
|          37|   Boro Zone| Brooklyn|155388|
|          42|   Boro Zone|Manhattan|143389|
|         231| Yellow Zone|Manhattan|135712|
|         234| Yellow Zone|Manhattan|132693|
|         161| Yellow Zone|Manhattan|128751|
+------------+------------+---------+------+



## 2. Manhattan 지역borough에서 발생한 하자 기준으로, zone별 하차건수(5천건 이상)출력

In [75]:
query = '''
    select zone_view.Borough, zone_view.service_zone, trip_view.DOLocationID, count (*) as ct
    from trip_view
    join zone_view
    on trip_view.DOLocationID = zone_view.LocationID
    group by zone_view.Borough, zone_view.service_zone, trip_view.DOLocationID
    having borough = 'Manhattan'
    and ct >= 5000
    order by ct desc
'''
spark.sql(query).show(5)

+---------+------------+------------+------+
|  Borough|service_zone|DOLocationID|    ct|
+---------+------------+------------+------+
|Manhattan| Yellow Zone|          79|147732|
|Manhattan|   Boro Zone|          42|136729|
|Manhattan|   Boro Zone|         244|127797|
|Manhattan| Yellow Zone|         161|121771|
|Manhattan| Yellow Zone|         234|118345|
+---------+------------+------------+------+
only showing top 5 rows



## 3. 각 서비스 존별로 승차와 하차건수 모두 계산한 뒤 총 이동건수(pick-up + drop-off)가 많은 순으로 정렬

In [70]:
#spark.stop()